За основу взят код с гитхаба
https://github.com/sl-iwi/auto_offers_parser/blob/master/app/auto_ru_parser.py  

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
from IPython.display import display
pd.options.display.max_columns = None

Посмотрим на тестовую выборку:

In [33]:
test = pd.read_csv('test.csv')

In [38]:
test.sample(10)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
4963,седан,TOYOTA,зелёный,бензин,1987.0,1.5 AT (94 л.с.),4.0,1991.0,SEDAN AUTOMATIC 1.5,автоматическая,1.5 LTR,94 N12,Мой первый автомобиль. На ходу. Старенький апп...,230000.0,"['[{""name"":""Комфорт"",""values"":[""Электростеклоп...",передний,Правый,Не требует ремонта,3 или более,Оригинал,Растаможен,5 лет и 4 месяца,4963
33182,внедорожник 5 дв.,VOLKSWAGEN,бежевый,бензин,2016.0,2.0 AMT (180 л.с.) 4WD,5.0,2017.0,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,2.0 LTR,180 N12,"Не требует вложений, никаких проблем нет, все ...",39000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,2 года и 11 месяцев,33182
5796,седан,HONDA,серебристый,бензин,1993.0,1.9 MT (115 л.с.),4.0,1997.0,SEDAN MECHANICAL 1.9,механическая,1.9 LTR,115 N12,Салон: 9. Безопасность: 2. Мультимедиа: 4. Ком...,330000.0,"['[{""name"":""Салон"",""values"":[""Тонированные сте...",передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,7 месяцев,5796
3909,внедорожник 5 дв.,TOYOTA,белый,бензин,2012.0,2.7 AT (185 л.с.) 4WD,5.0,2013.0,ALLROAD_5_DOORS AUTOMATIC 2.7,автоматическая,2.7 LTR,185 N12,Возможен обмен на Ваш авто.,91000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,3909
29082,седан,AUDI,чёрный,бензин,1994.0,2.4 AT (165 л.с.),4.0,1998.0,SEDAN AUTOMATIC 2.4,автоматическая,2.4 LTR,165 N12,В продаже Ауди а4 \n1998 года \n2.4 автомат \n...,273000.0,[],передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,NaN,29082
9355,универсал 5 дв.,SUZUKI,чёрный,бензин,2004.0,1.6 AT (107 л.с.),5.0,2007.0,WAGON_5_DOORS AUTOMATIC 1.6,автоматическая,1.6 LTR,107 N12,Отличный японский автомобиль с небольшим пробе...,163000.0,[],передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,6 месяцев,9355
29362,седан,AUDI,серебристый,бензин,2000.0,1.8 AT (170 л.с.) 4WD,4.0,2001.0,SEDAN AUTOMATIC 1.8,автоматическая,1.8 LTR,170 N12,ЛОТ: 01183714\nАвтопрага МКАД\n\nЦену этого ав...,274604.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,29362
33508,седан,VOLVO,серебристый,бензин,2000.0,2.4 MT (170 л.с.),4.0,2003.0,SEDAN MECHANICAL 2.4,механическая,2.4 LTR,170 N12,Автомобиль не требует вложений и полностью гот...,203833.0,[],передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,33508
35186,минивэн,MERCEDES,синий,дизель,2003.0,L3 2.2d AT (109 л.с.),4.0,2005.0,MINIVAN AUTOMATIC 2.2,автоматическая,2.2 LTR,109 N12,Автомобиль в РФ в одних руках с 2008 года. Тех...,210000.0,"['[{""name"":""Прочее"",""values"":[""Защита картера""...",задний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,11 лет и 7 месяцев,35186
3400,внедорожник 5 дв.,AUDI,серый,дизель,2009.0,3.0d AT (245 л.с.) 4WD,5.0,2015.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,245 N12,"Продам отличный семейный автомобиль, собранный...",176059.0,"['[{""name"":""Салон"",""values"":[""Тонированные сте...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,4 года и 9 месяцев,3400


In [40]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35990 entries, 0 to 35989
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              35990 non-null  object 
 1   brand                 35990 non-null  object 
 2   color                 35990 non-null  object 
 3   fuelType              35990 non-null  object 
 4   modelDate             35990 non-null  float64
 5   name                  35990 non-null  object 
 6   numberOfDoors         35990 non-null  float64
 7   productionDate        35990 non-null  float64
 8   vehicleConfiguration  35990 non-null  object 
 9   vehicleTransmission   35990 non-null  object 
 10  engineDisplacement    35990 non-null  object 
 11  enginePower           35990 non-null  object 
 12  description           35990 non-null  object 
 13  mileage               35990 non-null  float64
 14  Комплектация          35990 non-null  object 
 15  Привод             

In [ ]:
filter_large = test['mileage'] < 5
test.loc[filter_large]

In [ ]:
test['Комплектация'][0]

In [ ]:
test['description'][0]

In [47]:
test.shape

(35990, 23)

In [34]:
test.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'id'],
      dtype='object')

Тестовая выборка состоит из двух блоков информации для парсинга:

Часть bodytype - mileage парсим в первую очередь

Во вторую очередь парсим следующие поля:

'description'
'Комплектация'
'Привод'
'Руль'
'Состояние'
'Владельцы'
'ПТС'
'Таможня'
'Владение'
'id'


### Парсинг:

In [ ]:
vendors_list = ['volkswagen', 'audi', 'bmw','toyota','mercedes','skoda','nissan','mitsubishi','volvo','honda','lexus','infiniti','suzuki']

In [3]:
def search_url(vendor):
    location = 'moskva'
    model = 'all'
    url = f"https://auto.ru/{location}/cars/{vendor}/{model}"
    return url


In [4]:
#Функция парсит данные с нескольких страниц
#%%timeit
# this func returns dataframe with offers.
list_autos = []
def get_autoru_offers_all(u):
#   list_autos = []
#    autos = pd.DataFrame(list_autos)
    
    response = requests.get(u)
    soup = BeautifulSoup(response.content.decode('utf-8'), "html.parser")

    module_list = soup.find("div", class_= "ListingCars-module__container ListingCars-module__list")
    children = module_list.findChildren("div", recursive=False)
    for child in children:
        metas = child.find_all("meta")
        offer = {}
        for meta in metas:
            key = meta.get('itemprop')
            value = meta.get('content')
            offer.update({key: value}) #parse parameters except mileage
        try:        
            miles = child.find("div", {'class': 'ListingItem-module__kmAge'})
            offer.update({'mileage': miles.get_text()}) #parse mileage
        except:
            miles = ''
                    
        list_autos.append(offer)

    next_page = soup.find('link', attrs={'rel': 'next'})
    if next_page is not None:
           # If there is "NEXT" button => continue parsing to the next page
        time.sleep(2)
        next_url = next_page['href']
        get_autoru_offers_all(next_url)
    
    autos = pd.DataFrame(list_autos)
    autos = autos.drop(['image', 'availability', 'priceCurrency'], 1)
    autos.dropna(axis=0, inplace=True) #drop rows with missed values
    autos.reset_index(inplace = True,drop = True)
    return autos

In [5]:
#Функция парсит данные из автомобильного обьявления
#%%timeit
#list_car = pd.DataFrame()
def get_autoru_car_info(adr):
    car_info = pd.DataFrame(columns = ['Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС', 'Таможня'])
#    list_car = pd.DataFrame()
    list_car = []
    response = requests.get(adr)
    soup = BeautifulSoup(response.content.decode('utf-8'), "html.parser")
    mod_list = soup.find("ul", class_="CardInfo")
    offer = {}
    offer.update({'url': adr})
    row_drive = mod_list.find("li", class_="CardInfo__row CardInfo__row_drive").text[6:]
    offer.update({'Привод': row_drive})
    row_wheel = mod_list.find("li", class_="CardInfo__row CardInfo__row_wheel").text[4:]
    offer.update({'Руль': row_wheel})
    row_state = mod_list.find("li", class_="CardInfo__row CardInfo__row_state").text[9:]
    offer.update({'Состояние': row_state})
    row_ownersCount = mod_list.find("li", class_="CardInfo__row CardInfo__row_ownersCount").text[9:]
    offer.update({'Владельцы': row_ownersCount})  
    row_pts = mod_list.find("li", class_="CardInfo__row CardInfo__row_pts").text[3:]
    offer.update({'ПТС': row_pts})  
    row_customs = mod_list.find("li", class_="CardInfo__row CardInfo__row_customs").text[7:]
    offer.update({'Таможня': row_customs}) 
    try:
        row_owningTime = mod_list.find("li", class_="CardInfo__row CardInfo__row_owningTime").text[8:]
    except:
        row_owningTime = ''
    offer.update({'Владение': row_owningTime})
    card_description = soup.find("div", class_="CardDescription__textInner").text
    offer.update({'description': card_description})
   
    # парсинг блока с комплектацией
    cr = soup.find(string = re.compile("equipmentGroups"))
    index_start = cr.find('"equipmentGroups"') + 18
    index_end = cr.find('"owner_expenses"') - 2
    #print(cr[index_start:index_end])
    
    #в переменной names хранятся все ключи блока комплектации
    #names = soup.find_all("div", class_="CardComplectation-module__CardComplectation__itemName")

    offer.update({'Комплектация': cr[index_start:index_end]})
    
    list_car.append(offer)
        
    car_info = pd.DataFrame(list_car)
    
    return car_info

In [6]:
#функция принимает на вход марку авто и далее парсит сначала перечень обьявлений из поисковой выдачи, а затем для каждого обьявления по url парсит данные из конкретного обьявления
def parsing_func(brand):
    
    u = search_url(brand)
    
    train_set = get_autoru_offers_all(u)

    print(train_set.shape)

    counter = train_set['url'].shape[0]
    inside_list=[]
    for i in range(0, counter-1):
        try:
            inside_list.append(get_autoru_car_info(train_set['url'][i]))
        except:
            train_set['url'][i] = ''
    additional_data = pd.concat(inside_list, axis = 0, join = 'outer', ignore_index = False)
    train_set = train_set.merge(additional_data, how='left')
    print(train_set.shape)
    return train_set

In [19]:
#vw
list_autos = []
train_vw = parsing_func('volkswagen')
train_vw.to_csv('train_vw.csv')

(3663, 15)
(3813, 24)


In [22]:
#audi
list_autos = []
train_audi = parsing_func('audi')
train_audi.to_csv('train_audi.csv')

(3663, 15)
(3775, 24)


In [17]:
#bmw
list_autos = []
train_bmw = parsing_func('bmw')
train_bmw.to_csv('train_bmw.csv')

(3663, 15)
(3703, 24)


In [16]:
#toyota
list_autos = []
train_toyota = parsing_func('toyota')
train_toyota.to_csv('train_toyota.csv')

(3663, 15)
(3937, 24)


In [15]:
#mercedes
list_autos = []
train_merc = parsing_func('mercedes')
train_merc.to_csv('train_mercedes.csv')

(3663, 15)
(3713, 24)


In [14]:
#skoda
list_autos = []
train_skoda = parsing_func('skoda')
train_skoda.to_csv('train_skoda.csv')

(3663, 15)
(3759, 24)


In [13]:
#nissan
list_autos = []
train_nissan = parsing_func('nissan')
train_nissan.to_csv('train_nissan.csv')

(3663, 15)
(3799, 24)


In [12]:
#mitsubishi
list_autos = []
train_mit = parsing_func('mitsubishi')
train_mit.to_csv('train_mit.csv')

(3663, 15)
(3811, 24)


In [11]:
#volvo
list_autos = []
train_volvo = parsing_func('volvo')
train_volvo.to_csv('train_volvo.csv')

(2417, 15)
(2455, 24)


In [10]:
#honda
list_autos = []
train_honda = parsing_func('honda')
train_honda.to_csv('train_honda.csv')

(1801, 15)
(1843, 24)


In [9]:
#lexus
list_autos = []
train_lexus = parsing_func('lexus')
train_lexus.to_csv('train_lexus.csv')

(2222, 15)
(2234, 24)


In [8]:
#infiniti
list_autos = []
train_inf = parsing_func('infiniti')
train_inf.to_csv('train_inf.csv')

(1654, 15)
(1668, 24)


In [7]:
#suzuki
list_autos = []
train_suzuki = parsing_func('suzuki')
train_suzuki.to_csv('train_suzuki.csv')

(1136, 15)
(1170, 24)


In [26]:
train_vw = pd.read_csv('train_vw.csv')
train_audi = pd.read_csv('train_audi.csv')
train_bmw = pd.read_csv('train_bmw.csv')
train_toyota = pd.read_csv('train_toyota.csv')
train_merc = pd.read_csv('train_mercedes.csv')
train_skoda = pd.read_csv('train_skoda.csv')
train_nissan = pd.read_csv('train_nissan.csv')
train_mit = pd.read_csv('train_mit.csv')
train_volvo = pd.read_csv('train_volvo.csv')
train_honda = pd.read_csv('train_honda.csv')
train_lexus = pd.read_csv('train_lexus.csv')
train_inf = pd.read_csv('train_inf.csv')
train_suzuki = pd.read_csv('train_suzuki.csv')

In [27]:
df = pd.concat([train_vw, train_audi, train_bmw,train_toyota,train_merc,train_skoda, train_nissan, train_mit, train_volvo, train_honda, train_lexus, train_inf, train_suzuki], axis = 0, join = 'outer', ignore_index = False)

In [28]:
df['brand'].value_counts()

TOYOTA        3937
VOLKSWAGEN    3813
MITSUBISHI    3811
NISSAN        3799
AUDI          3775
SKODA         3759
MERCEDES      3713
BMW           3703
VOLVO         2455
LEXUS         2234
HONDA         1843
INFINITI      1668
SUZUKI        1170
Name: brand, dtype: int64

In [30]:
df = df.drop('Unnamed: 0', axis = 1)

In [31]:
df.head()

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,url,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,description,Комплектация
0,внедорожник 5 дв.,VOLKSWAGEN,белый,бензин,2016,2.0 AMT,5,2018,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,1890000,https://auto.ru/cars/used/sale/volkswagen/tigu...,2.0 LTR,180 N12,37 000 км,полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,"Продаю свой автомобиль , в дтп не участвовал ,...","[{""name"":""Салон"",""values"":[""Отделка кожей рыча..."
1,внедорожник 5 дв.,VOLKSWAGEN,чёрный,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,2449999,https://auto.ru/cars/used/sale/volkswagen/toua...,3.0 LTR,245 N12,99 999 км,полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,3 года и 6 месяцев,2016 г. По факту один хозяин (в ПТС 2). Пробег...,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",..."
2,внедорожник 5 дв.,VOLKSWAGEN,чёрный,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,2449999,https://auto.ru/cars/used/sale/volkswagen/toua...,3.0 LTR,245 N12,99 999 км,полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,3 года и 6 месяцев,2016 г. По факту один хозяин (в ПТС 2). Пробег...,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",..."
3,внедорожник 5 дв.,VOLKSWAGEN,синий,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,2550000,https://auto.ru/cars/used/sale/volkswagen/toua...,3.0 LTR,245 N12,46 000 км,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,3 года и 10 месяцев,"Автомобиль в отличном состоянии, R-LineОригина...","[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",..."
4,внедорожник 5 дв.,VOLKSWAGEN,синий,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,2550000,https://auto.ru/cars/used/sale/volkswagen/toua...,3.0 LTR,245 N12,46 000 км,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,3 года и 10 месяцев,"Автомобиль в отличном состоянии, R-LineОригина...","[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",..."


In [32]:
df.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'price', 'url', 'engineDisplacement',
       'enginePower', 'mileage', 'Привод', 'Руль', 'Состояние', 'Владельцы',
       'ПТС', 'Таможня', 'Владение', 'description', 'Комплектация'],
      dtype='object')

In [35]:
cols = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement',
       'enginePower','description', 'mileage', 'Комплектация','Привод', 'Руль', 'Состояние', 'Владельцы',
       'ПТС', 'Таможня', 'Владение', 'price', 'url']

In [52]:
df = df[cols]
df.head(10)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,url
0,внедорожник 5 дв.,VOLKSWAGEN,белый,бензин,2016,2.0 AMT,5,2018,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,2.0 LTR,180 N12,"Продаю свой автомобиль , в дтп не участвовал ,...",37 000 км,"[{""name"":""Салон"",""values"":[""Отделка кожей рыча...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,1890000,https://auto.ru/cars/used/sale/volkswagen/tigu...
1,внедорожник 5 дв.,VOLKSWAGEN,чёрный,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,245 N12,2016 г. По факту один хозяин (в ПТС 2). Пробег...,99 999 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,3 года и 6 месяцев,2449999,https://auto.ru/cars/used/sale/volkswagen/toua...
2,внедорожник 5 дв.,VOLKSWAGEN,чёрный,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,245 N12,2016 г. По факту один хозяин (в ПТС 2). Пробег...,99 999 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,3 года и 6 месяцев,2449999,https://auto.ru/cars/used/sale/volkswagen/toua...
3,внедорожник 5 дв.,VOLKSWAGEN,синий,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,245 N12,"Автомобиль в отличном состоянии, R-LineОригина...",46 000 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,3 года и 10 месяцев,2550000,https://auto.ru/cars/used/sale/volkswagen/toua...
4,внедорожник 5 дв.,VOLKSWAGEN,синий,дизель,2014,3.0 AT,5,2016,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,245 N12,"Автомобиль в отличном состоянии, R-LineОригина...",46 000 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,3 года и 10 месяцев,2550000,https://auto.ru/cars/used/sale/volkswagen/toua...
5,внедорожник 5 дв.,VOLKSWAGEN,чёрный,дизель,2016,2.0 AMT,5,2019,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,2.0 LTR,150 N12,NaN,Новый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2279400,NaN
6,универсал 5 дв.,VOLKSWAGEN,чёрный,дизель,2000,1.9 MT,5,2001,WAGON_5_DOORS MECHANICAL 1.9,механическая,1.9 LTR,101 N12,Фольцваген пассат 2001 года в отличном состоян...,147 565 км,"[{""name"":""Прочее"",""values"":[""Защита картера""]}...",передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,247000,https://auto.ru/cars/used/sale/volkswagen/pass...
7,внедорожник 5 дв.,VOLKSWAGEN,чёрный,бензин,2002,3.2 AT,5,2003,ALLROAD_5_DOORS AUTOMATIC 3.2,автоматическая,3.2 LTR,220 N12,NaN,200 000 км,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400000,NaN
8,седан,VOLKSWAGEN,серебристый,бензин,1996,1.6 MT,4,1997,SEDAN MECHANICAL 1.6,механическая,1.6 LTR,101 N12,NaN,284 000 км,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145000,NaN
9,седан,VOLKSWAGEN,белый,бензин,2014,1.6 AT,4,2019,SEDAN AUTOMATIC 1.6,автоматическая,1.6 LTR,110 N12,NaN,Новый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,856000,NaN


In [43]:
df.shape

(39680, 24)

In [44]:
df_final = df.drop_duplicates(keep=False, inplace=False)
df_final.shape

(30930, 24)

In [53]:
df_final.head(10)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,url
0,внедорожник 5 дв.,VOLKSWAGEN,белый,бензин,2016,2.0 AMT,5,2018,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,2.0 LTR,180 N12,"Продаю свой автомобиль , в дтп не участвовал ,...",37 000 км,"[{""name"":""Салон"",""values"":[""Отделка кожей рыча...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,1890000,https://auto.ru/cars/used/sale/volkswagen/tigu...
5,внедорожник 5 дв.,VOLKSWAGEN,чёрный,дизель,2016,2.0 AMT,5,2019,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,2.0 LTR,150 N12,NaN,Новый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2279400,NaN
6,универсал 5 дв.,VOLKSWAGEN,чёрный,дизель,2000,1.9 MT,5,2001,WAGON_5_DOORS MECHANICAL 1.9,механическая,1.9 LTR,101 N12,Фольцваген пассат 2001 года в отличном состоян...,147 565 км,"[{""name"":""Прочее"",""values"":[""Защита картера""]}...",передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,247000,https://auto.ru/cars/used/sale/volkswagen/pass...
7,внедорожник 5 дв.,VOLKSWAGEN,чёрный,бензин,2002,3.2 AT,5,2003,ALLROAD_5_DOORS AUTOMATIC 3.2,автоматическая,3.2 LTR,220 N12,NaN,200 000 км,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400000,NaN
8,седан,VOLKSWAGEN,серебристый,бензин,1996,1.6 MT,4,1997,SEDAN MECHANICAL 1.6,механическая,1.6 LTR,101 N12,NaN,284 000 км,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145000,NaN
9,седан,VOLKSWAGEN,белый,бензин,2014,1.6 AT,4,2019,SEDAN AUTOMATIC 1.6,автоматическая,1.6 LTR,110 N12,NaN,Новый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,856000,NaN
10,хэтчбек 5 дв.,VOLKSWAGEN,серебристый,бензин,2003,1.6 AT,5,2007,HATCHBACK_5_DOORS AUTOMATIC 1.6,автоматическая,1.6 LTR,115 N12,"Автомобиль в хорошем состоянии. Чистый, не кур...",165 700 км,"[{""name"":""Безопасность"",""values"":[""Подушка без...",передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,285000,https://auto.ru/cars/used/sale/volkswagen/golf...
11,хэтчбек 5 дв.,VOLKSWAGEN,серый,бензин,2009,1.2 AMT,5,2010,HATCHBACK_5_DOORS ROBOT 1.2,роботизированная,1.2 LTR,105 N12,Машина в отличном состоянии работает все отлич...,209 000 км,"[{""name"":""Комфорт"",""values"":[""Электростеклопод...",передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,369000,https://auto.ru/cars/used/sale/volkswagen/golf...
12,универсал 5 дв.,VOLKSWAGEN,чёрный,бензин,2008,1.4 AMT,5,2011,WAGON_5_DOORS ROBOT 1.4,роботизированная,1.4 LTR,122 N12,NaN,150 000 км,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,475000,NaN
13,седан,VOLKSWAGEN,серебристый,бензин,2010,1.4 AMT,4,2014,SEDAN ROBOT 1.4,роботизированная,1.4 LTR,122 N12,"Все автомобили, представленные в продаже, прох...",89 000 км,"[{""name"":""Прочее"",""values"":[""Защита картера""]}...",передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,499000,https://auto.ru/cars/used/sale/volkswagen/jett...


In [46]:
df_final['brand'].value_counts()

BMW           3140
VOLKSWAGEN    3103
NISSAN        3035
AUDI          2895
SKODA         2859
TOYOTA        2799
MITSUBISHI    2640
MERCEDES      2335
VOLVO         2314
HONDA         1717
LEXUS         1621
INFINITI      1441
SUZUKI        1031
Name: brand, dtype: int64

In [49]:
df_final.sample(10)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,url
2490,внедорожник 5 дв.,BMW,чёрный,дизель,2006,3.0 AT,5,2008,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,286 N12,Я собственник и продаю сам.Хороший надежный Х5...,265 150 км,"[{""name"":""Безопасность"",""values"":[""Система ста...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,3 года и 2 месяца,1199999,https://auto.ru/cars/used/sale/bmw/x5/10952902...
545,внедорожник 5 дв.,NISSAN,чёрный,бензин,2010,1.6 CVT,5,2013,ALLROAD_5_DOORS VARIATOR 1.6,вариатор,1.6 LTR,117 N12,◘ ПТС оригинал. Два комплекта ключей. В ДТП не...,73 700 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,599000,https://auto.ru/cars/used/sale/nissan/qashqai/...
908,седан,VOLVO,золотистый,дизель,2010,2.4 AT,4,2010,SEDAN AUTOMATIC 2.4,автоматическая,2.4 LTR,205 N12,Продаю отличный спорт седан. На полном приводе...,130 000 км,"[{""name"":""Прочее"",""values"":[""Защита картера""]}...",полный,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,1 год,850000,https://auto.ru/cars/used/sale/volvo/s60/10950...
2014,внедорожник 5 дв.,NISSAN,фиолетовый,бензин,2010,1.6 CVT,5,2013,ALLROAD_5_DOORS VARIATOR 1.6,вариатор,1.6 LTR,117 N12,Пройдены все ТО. Комплект летних шин в подарок...,77 000 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,1 год и 1 месяц,580000,https://auto.ru/cars/used/sale/nissan/juke/109...
301,внедорожник 5 дв.,INFINITI,коричневый,бензин,2008,3.7 AT,5,2010,ALLROAD_5_DOORS AUTOMATIC 3.7,автоматическая,3.7 LTR,333 N12,Тойота Центр Кунцево – официальный дилер!У нас...,231 868 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,947000,https://auto.ru/cars/used/sale/infiniti/fx/109...
618,внедорожник 5 дв.,VOLVO,синий,дизель,2006,2.4 AT,5,2012,ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,2.4 LTR,200 N12,200 л.с.129 пробег.Всё работает.Бежевая мягкая...,130 000 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,1 год и 4 месяца,1250000,https://auto.ru/cars/used/sale/volvo/xc90/1094...
923,внедорожник 5 дв.,BMW,серый,бензин,2017,2.0 AT,5,2020,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,2.0 LTR,184 N12,NaN,Новый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3302800,NaN
2185,внедорожник 5 дв.,TOYOTA,серый,бензин,2015,2.0 CVT,5,2017,ALLROAD_5_DOORS VARIATOR 2.0,вариатор,2.0 LTR,146 N12,"Продам RAV4.Куплена новой в 2017 году, в октяб...",10 193 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,2 года и 5 месяцев,1560000,https://auto.ru/cars/used/sale/toyota/rav_4/10...
3156,хэтчбек 5 дв.,VOLKSWAGEN,синий,бензин,1983,1.6 MT,5,1992,HATCHBACK_5_DOORS MECHANICAL 1.6,механическая,1.6 LTR,70 N12,"Машина не помыта, пардон..Была куплена в 2011 ...",280 000 км,"[{""name"":""Элементы экстерьера"",""values"":[""Стал...",передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,8 лет и 5 месяцев,60000,https://auto.ru/cars/used/sale/volkswagen/golf...
541,внедорожник 3 дв.,SUZUKI,жёлтый,бензин,2018,1.5 MT,3,2019,ALLROAD_3_DOORS MECHANICAL 1.5,механическая,1.5 LTR,102 N12,NaN,Новый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1680140,NaN


In [54]:
df_final.to_csv('train_all_columns.csv')

In [ ]:
DF = pd.read_csv('train_part_one.csv')

In [ ]:
DF['mileage'] = DF['mileage'].str.replace('<div class="ListingItem-module__kmAge">', '')
DF['mileage'] = DF['mileage'].str.replace('</div>', '')

In [ ]:
DF['mileage'] = DF['mileage'].str.replace('\xa0', '')
DF['mileage'] = DF['mileage'].str.replace('км', '')
DF['mileage'] = pd.to_numeric(DF['mileage'], errors='coerce')
DF['mileage'] = DF['mileage'].fillna(0)

In [ ]:
DF['mileage'].sample(10)